In [1]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

/home/spencersong/metamaterials_ai/src


In [2]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
from utils import rmse
import torch
import numpy as np
import pandas as pd
import altair as alt
import random
import itertools
data_folder = '../local_data'
work_folder = '../local_work'
configs_folder = '../configs'

In [91]:
def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

    watt_arg = torch.argmax(y_hat[0][2:])
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = max_speed * y_hat[0][0] # TODO call the scale
    y_final[1] = max_spacing * y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    y_final[0] = torch.round(y_final[0], decimals = -1)
    y_final[1] = torch.round(y_final[1], decimals = 1)
    y_final[2] = torch.round(y_final[2], decimals = 1)
    return y_final

def un_hot_encode(y_hat):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
    watt_arg = torch.argmax(y_hat[0][2:])
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = y_hat[0][0] # TODO call the scale
    y_final[1] = y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    y_final[0] = torch.round(y_final[0], decimals = -1)
    y_final[1] = torch.round(y_final[1], decimals = 1)
    y_final[2] = torch.round(y_final[2], decimals = 1)
    return y_final

# Inverse

In [80]:
inverse_ss_filepath = Path(f"{work_folder}/saved_best/I-1-res-ann-stainless.ckpt") #CHANGEME
inverse_inc_filepath = Path(f"{work_folder}/saved_best/I-1-res-ann-inconel.ckpt") #CHANGEME
print(inverse_ss_filepath)
print(inverse_inc_filepath)
if not Path.is_file(inverse_ss_filepath):
    raise Exception(f'Model file does not exist at {inverse_ss_filepath}!')
if not Path.is_file(inverse_inc_filepath):
    raise Exception(f'Model file does not exist at {inverse_inc_filepath}!')

../local_work/saved_best/I-1-res-ann-stainless.ckpt
../local_work/saved_best/I-1-res-ann-inconel.ckpt


In [81]:
def get_test_data(filepath):
    stain_data = torch.load(
                    Path(f"{data_folder}/{filepath}"))#["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
    wave_data = stain_data["interpolated_wavelength"]
    emiss_data = stain_data["interpolated_emissivity"]
    laser_params = stain_data["laser_params"]#.keys()#["wavelenght"]
    #print(wave_test.max(1), wave_test.min(1)) #2.5002, 11.9479
    #print(emiss_test.max(1)[0])#.max(1), emiss_test.min(1)) #0, 1
    wave_test = wave_data[round(len(wave_data) * .9):]  
    emiss_test = emiss_data[round(len(wave_data) * .9):]
    return wave_test, emiss_test, laser_params
#print(max_wave, min_wave)

In [82]:
# pass test data into inverse model, get predicted parameters
from src.models import InverseModel
i_model = InverseModel.load_from_checkpoint(inverse_ss_filepath, strict=False)
i_model.eval()

wave_test, emiss_test, laser_params = get_test_data('stainless-steel-revised-shuffled.pt')

test_y_hats = []#torch.empty(size = len(emiss_test))
for emiss in emiss_test:
    with torch.no_grad():
        test_y_hats.append(i_model(emiss.reshape(1, 800)))

In [92]:
laser_test = laser_params
max_speed, max_spacing = laser_test.max(0)[0][0].item(), laser_test.max(0)[0][1].item()
min_speed, min_spacing = laser_test.min(0)[0][0].item(), laser_test.min(0)[0][1].item()
laser_output = [denormalize_decode_result(x.reshape(1,14), max_speed, max_spacing, min_speed, min_spacing) for x in laser_test]

# denormalize_decode_result(laser_test[3].reshape(1,14),max_speed, max_spacing, min_speed, min_spacing)

# Direct Model

In [55]:
from src.models import DirectModel
direct_filepath = Path(""
    f"{work_folder}/saved_best/D-1-res-ann-stainless.ckpt") #CHANGEME
print(direct_filepath)
if not Path.is_file(direct_filepath):
    raise Exception(f'Model file does not exist at {direct_filepath}!')

../local_work/saved_best/D-1-res-ann-stainless.ckpt


In [56]:
d_model = DirectModel.load_from_checkpoint(direct_filepath)
d_model.eval()
emiss_y_hats = []
for x in test_y_hats:
    with torch.no_grad():
        emiss_y_hats.append(d_model(x.reshape(1,14)))

In [95]:
#calculate rmse
pred_rmse_vals = []
columns = ["wavelength", "emissivity", "prediction", "example_number", "laser parameters"]
viz_coords = pd.DataFrame(columns = columns)
for i in range(9):#len(emiss_test)):
    pred_rmse_vals.append(rmse(emiss_y_hats[i], emiss_test[i]).item())
    for j in range(len(wave_test[i].tolist())):
        viz_coords.append([wave_test[i].tolist()[j], emiss_test[i].tolist()[j], "True", i, laser_output[i].tolist()])
        viz_coords.append([wave_test[i].tolist()[j], emiss_y_hats[i].flatten().tolist()[j], "Prediction", i, laser_output[i].tolist()])
stdev = np.std(pred_rmse_vals)
print(f'Direct Filepath: {direct_filepath}')
print(f'Inverse Filepath: {inverse_ss_filepath}')
print(f'SD of RMSEs on test data: {stdev}')


/home/spencersong/metamaterials_ai/src/utils/utils.py:43: UserWarning: Using a target size (torch.Size([800])) that is different to the input size (torch.Size([1, 800])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return (torch.nn.functional.mse_loss(pred, target) + epsilon).sqrt()
/tmp/ipykernel_1219298/1076802862.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  viz_coords.append([wave_test[i].tolist()[j], emiss_test[i].tolist()[j], "True", i, laser_output[i].tolist()])
/tmp/ipykernel_1219298/1076802862.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  viz_coords.append([wave_test[i].tolist()[j], emiss_y_hats[i].flatten().tolist()[j], "Prediction", i, laser_output[i].tolist()])
/tmp/ipykernel_1219298/1076802862.py:8: FutureWarning: The frame.ap

In [ ]:
alt.Chart(viz_coords).mark_line().encode(
    x='wavelength',
    y='emissivity'
    legend=
).facet(facet = 'example_number:N')

In [ ]:
#visualize predictions vs original
